<a href="https://colab.research.google.com/github/cbonnin88/EDA_Projects/blob/main/Compensation_Analysis_FR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [240]:
import re
import polars as pl
import plotly.express as px
import plotly.io as pio

In [176]:
paystub_text = """
| BULLETIN DE PAIE |
| Période : 01/06/2024 au 30/06/2024 | | Date de paiement : 30/06/2024 |
EMPLOYEUR
Raison Sociale
	Tech Solutions SARL
	Adresse
	123 Rue de l'Innovation, 75012 Paris, France
	SIRET
	123 456 789 00010
	Code NAF (APE)
	6202A - Conseil en systèmes et logiciels informatiques
	URSSAF
	Île-de-France
	SALARIÉ
Nom et Prénom
	DUBOIS, Marie
	Adresse
	45 Avenue du Bloit, 75013 Paris, France
	N° de Sécurité Sociale
	2 85 01 75 123 456 78
	Emploi
	Analyste de Rémunération
	Classification
	Cadre, Coefficient 350
	Date d'entrée
	15/01/2022
	Ancienneté
	2 ans, 5 mois
	ÉLÉMENTS DE RÉMUNÉRATION
Libellé
	Base
	Taux
	Montant
Salaire de base
	151,67 h
	25,00 €
	3 791,75 €
Prime d'objectif


	500,00 €
Salaire Brut


	4 291,75 €
COTISATIONS ET CONTRIBUTIONS SOCIALES
Libellé	Base de Calcul	Taux Salarial	Part Salariale	Taux Patronal	Part Patronale
SANTÉ
Sécurité sociale - Maladie, Maternité
	4 291,75 €
	-
	-
	7,00%
	300,42 €
Complémentaire Incapacité Invalidité Décès
	4 291,75 €
	0,70%
	30,04 €
	0,80%
	34,33 €
Complémentaire Santé
	-
	-
	50,00 €
	-
	50,00 €
ACCIDENTS DU TRAVAIL
Accidents du travail - Maladies professionnelles
	4 291,75 €
	-
	-
	1,20%
	51,50 €
RETRAITE
Sécurité Sociale Plafonnée
	3 864,00 €
	6,90%
	266,62 €
	8,55%
	330,37 €
Sécurité Sociale Déplafonnée
	4 291,75 €
	0,40%
	17,17 €
	1,90%
	81,54 €
Complémentaire Tranche 1
	3 864,00 €
	3,15%
	121,72 €
	4,72%
	182,38 €
Complémentaire Tranche 2
	427,75 €
	8,64%
	36,96 €
	12,95%
	55,40 €
FAMILLE
Allocations familiales
	4 291,75 €
	-
	-
	3,45%
	148,07 €
CHÔMAGE
Assurance chômage
	4 291,75 €
	-
	-
	4,05%
	173,82 €
APEC
	4 291,75 €
	0,024%
	1,03 €
	0,036%
	1,54 €
AUTRES CONTRIBUTIONS
CSG déductible de l'impôt sur le revenu
	4 205,92 €
	6,80%
	286,00 €
	-
	-
CSG/CRDS non déductible
	4 205,92 €
	2,90%
	121,97 €
	-
	-
Total des retenues salariales


	931,51 €
NET À PAYER
Libellé
	Montant
Salaire Brut
	4 291,75 €
Total des retenues salariales
	- 931,51 €
Net à payer avant impôt sur le revenu
	3 360,24 €
IMPÔT SUR LE REVENU
Base du prélèvement à la source
	3 238,27 €
Taux de prélèvement (non personnalisé)
	9,00%
Montant du prélèvement à la source
	- 291,44 €
Net payé
	3 068,80 €
"""

In [220]:
def parse_paystub(text):

  def clean_number(s):
   if s is None or s.strip() =='-' or s.strip() == '':
      return 0.0
   return float(s.strip().replace(' ','').replace('€','').replace(',','.'))


  patterns = {
      'salaire_brut':r'Salaire Brut\s+([\d\s,.]+)\s*€',
      'total_retenues':r'Total des retenues salariales\s+-\s*([\d\s,.]+)\s*€',
      'net_avant_impot':r'Net à payer avant impôt sur le revenu\s+([\d\s,.]+)\s*€',
      'impot':r'Montant du prélèvement à la source\s+-\s*([\d\s,.]+)\s*€',
      'net_paye':r'Net payé\s+([\d\s,.]+)\s*€'
  }

  extracted_data = {}
  for key,pattern in patterns.items():
    match = re.search(pattern,text)
    if match:
      extracted_data[key] = clean_number(match.group(1))

  contributions = []
  contribution_labels = [
        "Sécurité sociale - Maladie, Maternité",
        "Complémentaire Incapacité Invalidité Décès",
        "Complémentaire Santé",
        "Accidents du travail - Maladies professionnelles",
        "Sécurité Sociale Plafonnée",
        "Sécurité Sociale Déplafonnée",
        "Complémentaire Tranche 1",
        "Complémentaire Tranche 2",
        "Allocations familiales",
        "Assurance chômage",
        "APEC",
        "CSG déductible de l'impôt sur le revenu",
        "CSG/CRDS non déductible"
    ]

  for label in contribution_labels:
        pattern = re.compile(rf"^{re.escape(label)}\s*\n\s*(.*?)\n\s*(.*?)\n\s*(.*?)\n\s*(.*?)\n\s*(.*?)$", re.MULTILINE)
        match = pattern.search(text)

        if match:
            contributions.append({
                "Libellé": label,
                "Base de Calcul": clean_number(match.group(1)),
                "Taux Salarial": match.group(2).strip(),
                "Part Salariale": clean_number(match.group(3)),
                "Taux Patronal": match.group(4).strip(),
                "Part Patronale": clean_number(match.group(5))
            })


  extracted_data['contributions'] = contributions
  return extracted_data

pay_data = parse_paystub(paystub_text)

if pay_data is None:
    print("Fatal Error: The parse_paystub function failed and returned None.")
elif not pay_data.get('contributions'):
    print("Error: No contribution data was extracted. The DataFrame cannot be created.")
else:
    print("--- Extracted Pay Data (Summary) ---")
    for key, value in pay_data.items():
        if key != 'contributions':
            print(f"{key}: {value}")
    print("-" * 20)



df_paystub = pl.DataFrame(pay_data['contributions'])
df_paystub = df_paystub.with_columns(
    pl.when(pl.col('Libellé').str.contains('Santé|Maladie'))
      .then(pl.lit('Santé'))
      .when(pl.col('Libellé').str.contains('Accident'))
      .then(pl.lit('Accidents du Travail'))
      .when(pl.col('Libellé').str.contains('Retraite|Sociale|Complémentaire Tranche'))
      .then(pl.lit('Retraite'))
      .when(pl.col('Libellé').str.contains('familiales'))
      .then(pl.lit('Famille'))
      .when(pl.col('Libellé').str.contains('chômage|APEC'))
      .then(pl.lit('Chômage'))
      .when(pl.col('Libellé').str.contains('CSG|CRDS'))
      .then(pl.lit('Autres Contributions'))
      .otherwise(pl.lit('Autres Contributions'))
      .alias('Catégorie')

)

total_part_patronale = df_paystub['Part Patronale'].sum()
total_labor_cost = pay_data['salaire_brut'] + total_part_patronale

print('\n--- Compensation Analysis ---')
print(f'Salaire Brut: €{pay_data["salaire_brut"]:.2f}')
print(f'Total Cotisations Patronales: €{total_part_patronale:.2f}')
print(f'Coût Total Employeur: €{total_labor_cost:.2f}')

net_to_cost_ratio = (pay_data['net_paye']/ total_labor_cost) * 100
print(f'Ratio Net Payé / Coût Total: {net_to_cost_ratio:.1f}%')

employer_contributions_by_cat = df_paystub.group_by('Catégorie').agg(
    pl.sum('Part Patronale').alias('Total_Part_Patronale')
).sort('Total_Part_Patronale',descending=True)


print('\nRépartition des Cotisations Patronales Par Catégorie')
display(employer_contributions_by_cat)

--- Extracted Pay Data (Summary) ---
salaire_brut: 4291.75
total_retenues: 931.51
net_avant_impot: 3360.24
impot: 291.44
net_paye: 3068.8
--------------------

--- Compensation Analysis ---
Salaire Brut: €4291.75
Total Cotisations Patronales: €1409.37
Coût Total Employeur: €5701.12
Ratio Net Payé / Coût Total: 53.8%

Répartition des Cotisations Patronales Par Catégorie


Catégorie,Total_Part_Patronale
str,f64
"""Retraite""",649.69
"""Santé""",401.92
"""Chômage""",175.36
"""Famille""",148.07
"""Autres Contributions""",34.33


# **Data Visualizations with Plotly**

In [221]:
waterfall_data = [
    {'measure':'relative','x':'Salaire Brut', 'y':pay_data['salaire_brut']},
    {'measure':'relative','x':'Cotisations Salariales','y':-pay_data['total_retenues']},
    {'measure':'total','x':'Net Avant Impôt','y':pay_data['net_avant_impot']},
    {'measure':'relative','x':'Impôt sur le Revenu','y':-pay_data['impot']},
    {'measure':'total','x':'Net Payé','y':pay_data['net_paye']}

]
waterfall_df = pl.DataFrame(waterfall_data)

In [222]:
fig_waterfall = px.bar(waterfall_df,
                       x='x',
                       y='y',
                       color ='measure',
                       labels = {'x': 'Etape', 'y':'Montant (€)'},
                       title = 'De Salaire Brut à net Payé',
                       text= [f'€{val:.2f}' for val in waterfall_df['y']],
                       color_discrete_sequence = px.colors.sequential.Viridis_r
                     )
fig_waterfall.update_traces(textposition='outside')
fig_waterfall.update_layout(showlegend=False)
fig_waterfall.show()

In [238]:
sunburst_data = {
        'ids': ['Coût Total Employeur', 'Salaire Brut', 'Cotisations Patronales'],
        'parents': ['', 'Coût Total Employeur', 'Coût Total Employeur'],
        'values': [total_labor_cost, pay_data.get('salaire_brut', 0), total_part_patronale]
    }

for row in employer_contributions_by_cat.iter_rows(named=True):
    if row['Total_Part_Patronale'] > 0:
      sunburst_data['ids'].append(row['Catégorie'])
      sunburst_data['parents'].append('Cotisations Patronales')
      sunburst_data['values'].append(row['Total_Part_Patronale'])

plot_df = pl.DataFrame(sunburst_data)

print("\n--- Sunburst Chart DataFrame (plot_df) ---")
display(plot_df)


--- Sunburst Chart DataFrame (plot_df) ---


ids,parents,values
str,str,f64
"""Coût Total Employeur""","""""",5701.12
"""Salaire Brut""","""Coût Total Employeur""",4291.75
"""Cotisations Patronales""","""Coût Total Employeur""",1409.37
"""Retraite""","""Cotisations Patronales""",649.69
"""Santé""","""Cotisations Patronales""",401.92
"""Chômage""","""Cotisations Patronales""",175.36
"""Famille""","""Cotisations Patronales""",148.07
"""Autres Contributions""","""Cotisations Patronales""",34.33


In [250]:
pio.renderers.default = 'colab'

fig_sunburst = px.sunburst(
        plot_df,
        names='ids',
        parents='parents',
        values='values',
        title='Répartition du Coût Total Employeur',
        branchvalues='total',
        color_discrete_sequence = px.colors.sequential.Plasma_r
    )

fig_sunburst.update_traces(textinfo='label+percent parent')
fig_sunburst.show()

In [253]:
contributions_by_type = df_paystub.group_by('Catégorie').agg(
    pl.sum('Part Salariale').alias('Salariale'),
    pl.sum('Part Patronale').alias('Patronale')
)

In [258]:
plot_df_grouped = contributions_by_type.melt(
    id_vars = 'Catégorie',
    value_vars = ['Salariale','Patronale'],
    variable_name='Type de Cotisation',
    value_name = 'Montant'
)

print('\n--- Group Bar Chart DataFrame (plot_df_grouped) ---')
display(plot_df_grouped)


--- Group Bar Chart DataFrame (plot_df_grouped) ---


Catégorie,Type de Cotisation,Montant
str,str,f64
"""Retraite""","""Salariale""",442.47
"""Famille""","""Salariale""",0.0
"""Chômage""","""Salariale""",1.03
"""Autres Contributions""","""Salariale""",438.01
"""Santé""","""Salariale""",50.0
"""Retraite""","""Patronale""",649.69
"""Famille""","""Patronale""",148.07
"""Chômage""","""Patronale""",175.36
"""Autres Contributions""","""Patronale""",34.33


In [263]:
fig_grouped_bar= px.bar(
    plot_df_grouped,
    x='Catégorie',
    y='Montant',
    color ='Type de Cotisation',
    barmode='group',
    title='Répartition des Cotisations par Type (Salariale vs Patronale)',
    labels={'Montant':'Montant Total (€)','Catégorie':'Catégorie de Cotisation'},
    text_auto = '.2f',
    color_discrete_map={
            'Salariale': '#90CAF9',
            'Patronale': '#77DD77'
        }
)

fig_grouped_bar.show()